In [ ]:
import random

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchinfo import summary

from model import BaseModel # TODO: モデルは自作する
import torch_utils

In [ ]:
BATCH_SIZE = 256
CPU_NUM = 2

df = pd.read_csv('')
dataset = Dataset(df)

train, val, test = random_split(dataset, [0.6, 0.2, 0.2])

# seed固定用
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

train_loader = DataLoader(
    dataset=train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=CPU_NUM,
    pin_memory=True,
    worker_init_fn=seed_worker,
    generator=g,
)

val_loader = DataLoader(
    dataset=val,
    batch_size=BATCH_SIZE,
    num_workers=CPU_NUM,
    pin_memory=True,
    worker_init_fn=seed_worker,
    generator=g,
)

test_loader = DataLoader(
    dataset=test,
    batch_size=BATCH_SIZE,
    num_workers=CPU_NUM,
    pin_memory=True,
    worker_init_fn=seed_worker,
    generator=g,
)

loader_dict = {
    'train': train_loader,
    'val': val_loader,
    'test': test_loader
}

In [ ]:
torch_utils.torch_seed()

EPOCHS = 50
history = []

device = torch_utils.check_gpu()

model = BaseModel().to(device)
# モデル詳細を表示したい場合は以下のコメントを解除し、入力のshapeを指定し使用。
# print(summary(
#     model,
#     input_size=() # TODO: ここを編集
# ))

optimizer = optim.Adam(model.parameters())

criterion = nn.MSELoss()

history = torch_utils.fit(model, optimizer, criterion, EPOCHS, loader_dict, device, history)